In [ ]:
#import required libraries for the clump analysis
import numpy as np
import pandas as pd
import scipy.spatial
import math as math

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors
from matplotlib.ticker import ScalarFormatter

from astropy.stats import jackknife_resampling
from astropy.stats import jackknife_stats

#define a class that we later use for the formatting of X and Y axis in the plot.
class ScalarFormatterForceFormat(ScalarFormatter):
    def _set_format(self):
        self.format = "%1.1f"

#define function for the Jackknife analysis
def JK_stats(array):
    arr = array.to_numpy()
    estimate, bias, error, conf_internval = jackknife_stats(arr, np.mean, 0.95)
    return estimate, bias, error, conf_internval

#reading in the data from .dat file and create a dataframe 
#R_pos and Z_pos are in km.
df = pd.read_csv("../std_def_final/particles_data_Z0.dat", 
                     sep = "\s+", 
                     usecols = [0, 1, 2, 3, 4, 5, 6, 7, 8], 
                     names = ['X_Fe', 'X_Ni', 'X_Mn', 'X_Cr', 'X_Ti', 'X_Cr_to_Fe', 'X_Ti_to_Fe', 'R_pos', 'Z_pos'])

#sort dataframe in descending order of Cr/Fe ratio such that first element in dataframe has highest Cr/Fe ratio.
df.sort_values(by = ["X_Cr_to_Fe"], ascending = False, inplace = True)
#reset the index of the dataframe
df = df.reset_index(drop = True)

#assign seperate dateframe for z (Cr/Fe ratios), x (R-position), y (Z-position)
#please note that here x is cylindrical R-coordinates and y is cylindrical Z-coordinates of particles. 
Cr_to_Fe = df['X_Cr_to_Fe']
x = df['R_pos']
y = df['Z_pos']

#get n = length of the dataframe
n = len(df)

# empty lists to store the analyzed data.
clump_array = [] 
Ni_to_Fe_JKS = []
Mn_to_Fe_JKS = []
Cr_to_Fe_JKS = []
Ti_to_Fe_JKS = []

#first for loop to analyze all the particles 
for j in range(0,n):
    print(j)    
    p = Cr_to_Fe[j] #p is variable that is Cr/Fe ratio of the point(particle) that we are analyzing in the loop
                    #p begins from 0 that is first entry in Cr/Fe dataframe and it loop over n particles
             
    
    x0 = x[j]    # x0 is similar as above, the R-coord. of our particle that we are analyzing
    y0 = y[j]    # y0 is similar as above, the Z-coord. of our particle that we are analyzing

    points = list(zip(x,y)) # we create the list that has of pairs of R and Z coordinates of all particles
        
    ckdtree = scipy.spatial.cKDTree(points) # Scipy function to create KD Tree to lookup the nearest neighbour.
                                            # for more info please look up at 
                                            # https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.cKDTree.html
    
    # make sure these lists are empty before they are used again in the loop below
    clump_points = [] # used to store the data (distances and index) of the nearest neighbor points.
    clump_data = []   # used for creating a seperate dataframe for the particles identified in the clump.
       
        
    #next for loop that looks up into nearest neighbour of our particle 'p' from ckdtree created
    for increment in range(1,n+1):
        clump_points = ckdtree.query([x0,y0],increment) # ckdtree.query takes in two arguments 1st is x and y position of our 
                                                        # reference point or the point that we are analyzing (p) and 2nd argument is 
                                                        # number of nearest neighbours to consider 
                                                        # i.e. 1 = point itself, 2 = point itself and nearest point to it and so on. 
                                                        # this returns list of two arrays.
                                                        # 1st array is the distances of nearest neighbor points from our referance point.
                                                        # 2nd array is the index of those nearest neighbor points.    
                                                        # to read more - https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.cKDTree.query.html#scipy.spatial.cKDTree.query
            
        clump_data = df.iloc[clump_points[1]]  # here we use 2nd array of clump_points which contains the index of particles in our clump.
                                               # we locate these index on our orignal 'df' dataframe and filter those particles data as 
                                               # new dataframe called clump_data.
                                               # clump_data has same columns as df dataframe, but for our filtered particles.
            
        if (clump_data["X_Fe"].sum()) >= 0.07 * (df["X_Fe"].sum()):  # the condition for termination. 
                                                                         # if sum of X_fe for all the particles in the clump 
                                                                         # that we are indentifying, 
                                                                         # is > or = 0.07 of sum of X_fe in ejecta. loop terminates.
                
                print(clump_data['X_Fe'].sum()/df['X_Fe'].sum())         # print to verify.
    
                break # break the loop
        
    
    #At this point we have a data from our clump and we can begin the analysis using clump_data
    
    #Jacknife analysis of all the abundances that we report
    Fe_est, Fe_bias, Fe_err, Fe_interval = JK_stats(clump_data["X_Fe"])
    #Ni_est, Ni_bias, Ni_err, Ni_interval = JK_stats(clump_data["X_Ni"])
    #Mn_est, Mn_bias, Mn_err, Mn_interval = JK_stats(clump_data["X_Mn"])
    Cr_est, Cr_bias, Cr_err, Cr_interval = JK_stats(clump_data["X_Cr"])
    #Ti_est, Ti_bias, Ti_err, Ti_interval = JK_stats(clump_data["X_Ti"])

    #print out the ratios of our Jack-knife analysis. such as ratios of mean, error and confidence intervals.
    print("Est. :" , Cr_est/Fe_est)
    print("Error :", Cr_err/Fe_err)
    print("Conf. interval :", Cr_interval/Fe_interval)
    
    #print("Est. :" , Ni_est/Fe_est, Mn_est/Fe_est, Cr_est/Fe_est, Ti_est/Fe_est)
    #print("Error :" , Ni_err/Fe_err, Mn_err/Fe_err, Cr_err/Fe_err, Ti_err/Fe_err)
    #print("Conf. interval :" , Ni_interval/Fe_interval, Mn_interval/Fe_interval, Cr_interval/Fe_interval, Ti_interval/Fe_interval)
    
    fst_pt_r = clump_data["R_pos"].iloc[0]        # R-coord. of our reference point
    fst_pt_z = clump_data["Z_pos"].iloc[0]        # Z-coord. of our referance point
                
    lst_pt_r = clump_data["R_pos"].iloc[-1]       # R-ccord. of last nearest neighbor point that was added to the clump 
    lst_pt_z = clump_data["Z_pos"].iloc[-1]       # Z-ccord. of last nearest neighbor point that was added to the clump
                
    r_dist = abs(lst_pt_r - fst_pt_r)                   # distance in R-direction between two points 
    z_dist = abs(lst_pt_z - fst_pt_z)                   # distance in Z-direction between two points
    radius_dist = math.sqrt((r_dist**2) + (z_dist**2))  # actual distance between two points
    print("%.5e" %fst_pt_r,"km", "%.5e" %fst_pt_z, "km", "%.5e" %radius_dist, "km") # print center-point's coordinates and the radius.
    
    total_Fe = np.sum(clump_data["X_Fe"])
    total_Cr = np.sum(clump_data["X_Cr"])
    mean_Fe = np.mean(clump_data["X_Fe"])   
    mean_Cr = np.mean(clump_data["X_Cr"])   
    clump_Cr_to_Fe = mean_Cr/mean_Fe 
    
    print("total Fe in the clump: ", "%.5f" %total_Fe) #print sum of mass fractions of Fe in the clump particles
    print("total Cr in the clump: ", "%.5f" %total_Cr) #print sum of mass fractions of Cr in the clump particles
    print("mean Fe in the clump: ", "%.5f" %mean_Fe)  #print mean_Fe
    print("mean Cr in the clump: ","%.5f" %mean_Cr)   #print mean_Cr
    print("Cr/Fe in the clump : ", "%.5f" %clump_Cr_to_Fe) #print ratio of Cr/Fe
    print() #empty print to seperate consicutive loop's prints
    
    #store the important data into array and use this at very end to plot
    clump_array.append([total_Fe,
                        total_Cr,
                        mean_Fe, 
                        mean_Cr, 
                        clump_Cr_to_Fe, 
                        fst_pt_r, 
                        fst_pt_z, 
                        radius_dist]) 

#create dataframe from the list clump_array since it is easier to work with.
clumps_df = pd.DataFrame(clump_array, 
                         columns = ['total_Fe',
                                    'total_Cr',
                                    'mean_Fe', 
                                    'mean_Cr', 
                                    'clump_Cr_to_Fe', 
                                    'fst_pt_r', 
                                    'fst_pt_z', 
                                    'radius_dist'], 
                         dtype = float)

#sort it in descending order of clump_Cr_to_Fe column
clumps_df.sort_values(by = ["clump_Cr_to_Fe"], ascending = False, inplace = True)

#print the dataframe
print(clumps_df.to_string())

clumps_df = clumps_df.reset_index(drop = True)

             

In [ ]:
#print(clumps_df.to_string())

#plotting begins here.
#cent is used for setting up domain size
cent = 2.1e4

#define figure and axes. 
fig, ax = plt.subplots(figsize = (5,10))

size = df["X_Cr_to_Fe"]

#plotting the scatter plot 
sp = ax.scatter(df['R_pos'], df['Z_pos'], 
                s = (60 * ((size - size.min())/(size.max() - size.min()))) + 0.5,
                c = size, cmap = 'RdBu', norm = colors.LogNorm(vmax = 1.0e5, vmin = 1.0e-2))

#few edits to the plot such as domain size, formtting of x and y axis, labels, and background color etc.
ax.set_xlim(0,2*cent)
ax.set_ylim(-2*cent,2*cent)
ax.tick_params(labelsize = 16)
yfmt = ScalarFormatterForceFormat()
yfmt.set_powerlimits((0,0))
ax.yaxis.set_major_formatter(yfmt)
ax.xaxis.set_major_formatter(yfmt)
ax.ticklabel_format(axis = 'both', style = 'sci', useMathText= True, scilimits = [0,0])
ax.tick_params(axis='both',labelsize = 20)
ax.xaxis.get_offset_text().set_fontsize(20)
ax.yaxis.get_offset_text().set_fontsize(20)
ax.set_xlabel('r (km)', fontsize = 20)
ax.set_ylabel('z (km)', fontsize = 20)
ax.set_facecolor('black')

#draw the circle on the plot with coordinates of center point 
#of first element in clumps_df dataframe and radius.

ax.add_patch(plt.Circle((clumps_df['fst_pt_r'].iloc[0], clumps_df['fst_pt_z'].iloc[0]), 
                                   clumps_df['radius_dist'].iloc[0], fill = False, linewidth = 2.0,color ='white'))

#draw colorbar below the plot and add labels to it.
bar = ax.inset_axes([0.0,-0.15,1,0.02])
c_bar = fig.colorbar(sp, cax = bar, orientation = 'horizontal', extend = 'min')
c_bar.ax.tick_params(labelsize = 20)
c_bar.set_label(label = r'$\mathtt{\frac{X_{Cr}}{X_{Fe}}}$', size = 26)

#show the plot
plt.show()

#save figure
fig.savefig("Scatter_plot_with_clump.png", bbox_inches = 'tight')






In [ ]:
#   Fe_est, Fe_bias, Fe_err, Fe_interval = JK_stats(clump_data["X_Fe"])
#    Ni_est, Ni_bias, Ni_err, Ni_interval = JK_stats(clump_data["X_Ni"])
#    Mn_est, Mn_bias, Mn_err, Mn_interval = JK_stats(clump_data["X_Mn"])
#    Cr_est, Cr_bias, Cr_err, Cr_interval = JK_stats(clump_data["X_Cr"])
#    Ti_est, Ti_bias, Ti_err, Ti_interval = JK_stats(clump_data["X_Ti"])
            
#    print("Est. :" , Ni_est/Fe_est, Mn_est/Fe_est, Cr_est/Fe_est, Ti_est/Fe_est)
#    print("Error :" , Ni_err/Fe_err, Mn_err/Fe_err, Cr_err/Fe_err, Ti_err/Fe_err)
#    print("Conf. interval :" , Ni_interval/Fe_interval, Mn_interval/Fe_interval, Cr_interval/Fe_interval, Ti_interval/Fe_interval)
    
#    clump_array.append([len(clump_data), 
#                        np.mean(clump_data['X_Fe']),
#                        np.mean(clump_data['X_Ni']),
#                        np.mean(clump_data['X_Mn']),
#                        np.mean(clump_data['X_Cr']),
#                        np.mean(clump_data['X_Ti']),
#                        (np.mean(clump_data['X_Ni']))/(np.mean(clump_data['X_Fe'])),
#                        (np.mean(clump_data['X_Mn']))/(np.mean(clump_data['X_Fe'])),
#                        (np.mean(clump_data['X_Cr']))/(np.mean(clump_data['X_Fe'])),
#                        (np.mean(clump_data['X_Ti']))/(np.mean(clump_data['X_Fe']))]) 
    
#    Ni_to_Fe_JKS.append([(Ni_est/Fe_est), 
#                        (Ni_err/Fe_err), 
#                        (Ni_interval/Fe_interval), 
#                        ((Ni_interval[1]/Fe_interval[1]) - Ni_est/Fe_est),
#                        ((Ni_interval[0]/Fe_interval[0]) - Ni_est/Fe_est)])
    
#    Mn_to_Fe_JKS.append([(Mn_est/Fe_est), 
#                        (Mn_err/Fe_err), 
#                        (Mn_interval/Fe_interval), 
#                        ((Mn_interval[1]/Fe_interval[1]) - Mn_est/Fe_est),
#                        ((Mn_interval[0]/Fe_interval[0]) - Mn_est/Fe_est)])
    
#    Cr_to_Fe_JKS.append([(Cr_est/Fe_est), 
#                        (Cr_err/Fe_err), 
#                        (Cr_interval/Fe_interval), 
#                        ((Cr_interval[1]/Fe_interval[1]) - Cr_est/Fe_est),
#                        ((Cr_interval[0]/Fe_interval[0]) - Cr_est/Fe_est)])
    
#    Ti_to_Fe_JKS.append([(Ti_est/Fe_est), 
#                        (Ti_err/Fe_err), 
#                        (Ti_interval/Fe_interval), 
#                        ((Ti_interval[1]/Fe_interval[1]) - Ti_est/Fe_est),
#                        ((Ti_interval[0]/Fe_interval[0]) - Ti_est/Fe_est)])

    
#clumps_df = pd.DataFrame(clump_array, 
#                         columns = ['parts', 'mean_fe', 'mean_ni', 'mean_mn', 'mean_cr', 'mean_ti', 'ni_to_fe', 'mn_to_fe', 'cr_to_fe', 'ti_to_fe'], 
#                         dtype = float)

#clumps_df.sort_values(by = 'cr_to_fe', ascending = False, inplace = True)
#clumps_df.to_csv('../std_def_final/clumps_details_Z30.dat', sep = ' ',header = True, index = True, float_format = '%g')
#print(clumps_df.to_string())

#Ni_df = pd.DataFrame(Ni_to_Fe_JKS,columns = ['mean', 'error', 'interval', 'upper_bar', 'lower_bar'], dtype = float)
#Ni_df = Ni_df.reindex(clumps_df.index)
#Ni_df.to_csv('../std_def_final/clumps_details_Ni_Z30.dat', sep = ' ',header = True, index = True, float_format = '%g')

#Mn_df = pd.DataFrame(Mn_to_Fe_JKS,columns = ['mean', 'error', 'interval', 'upper_bar', 'lower_bar'], dtype = float)
#Mn_df = Mn_df.reindex(clumps_df.index)
#Mn_df.to_csv('../std_def_final/clumps_details_Mn_Z30.dat', sep = ' ',header = True, index = True, float_format = '%g')

#Cr_df = pd.DataFrame(Cr_to_Fe_JKS,columns = ['mean', 'error', 'interval', 'upper_bar', 'lower_bar'], dtype = float)
#Cr_df = Cr_df.reindex(clumps_df.index)
#Cr_df.to_csv('../std_def_final/clumps_details_Cr_Z30.dat', sep = ' ',header = True, index = True, float_format = '%g')

#Ti_df = pd.DataFrame(Ti_to_Fe_JKS,columns = ['mean', 'error', 'interval', 'upper_bar', 'lower_bar'], dtype = float)
#Ti_df = Ti_df.reindex(clumps_df.index)
#Ti_df.to_csv('../std_def_final/clumps_details_Ti_Z30.dat', sep = ' ',header = True, index = True, float_format = '%g')


#Cr_df.sort_values(by = 'mean', ascending = False, inplace = True)
#print(Cr_df.to_string())

#clumps_df = clumps_df[clumps_df['cr_to_fe'] >= 0.097]
#clumps_df = clumps_df[clumps_df['cr_to_fe'] <= 0.117]
#clumps_df = clumps_df[clumps_df['ni_to_fe'] >= 0.15 ]
#clumps_df = clumps_df[clumps_df['ni_to_fe'] <= 0.25 ]

print()

In [ ]:
import numpy as np
from astropy.stats import jackknife_stats
# Draw 100 samples from the normal distribution
samples = np.random.randn(10)
# Compute the standard error using jackknife_stats
test_statistic, mean_jackknifed, bias, mean_std = jackknife_stats(samples, np.mean)
print(f"Sample mean (Jackknife): {mean_jackknifed}")
print(f"Standard error (Jackknife): {mean_std}")
# To compute the 95% confidence interval for the mean
confidence_level = 0.95
z_value = 1.96 # For a 95% confidence interval with a normal distribution
lower_bound = mean_jackknifed - z_value * mean_std
upper_bound = mean_jackknifed + z_value * mean_std
print(f"95% Confidence interval: ({lower_bound}, {upper_bound})")